-----------------------
# **Source**

Content from [Learning Python Design Patterns (Chetan Giridhar, 2016)](https://www.amazon.com.br/dp/B018XYKNOM/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1).

-----------------------

--------
# **Observer pattern**

- Is one kind of  *behavioral pattern*.
- In this pattern, an object (*subject*) maintains a list of all dependents (*obsevers*) so that the subject can notify all observers about changes that undergoes using any of the methods defined by the observer.

- In the world of distributed applications, multiple services interact with each other to perform a larger operation that a user wants to achieve. Services can perform multiple operations, but the operation they perform is directly or heavily dependent on the state of the objects of the service that it interacts with.


- Consider a use case for a user registration where the user service is responsible for user operations on the website and another service called "e-mail service" that observes the user's state and send e-mails for it depending on user's actions.
    - If there is a core service in some application on which many other services are dependent, the core service becomes the subject that has to be observed/monitored because of its possible changes. The observer role is responsible to *observe* (~~really?~~) this changes when they happen and then perform some action based on the observed result.
    
    
- In general, the main intentions of the observer pattern are:
    - It defines a one-to-many dependency between objects so that any change in one object will be notified to the other dependent objects automatically.
    - It encapsulates the core component of the subject.
    
    
- The main applications of this kind of pattern are common in the following scenarios:
    - Implementation of event service in distributed systems.
    - Framework for a news agency.
    - Stock market

--------------------------------
# **Implementation**

In [1]:
class Subject:
    def __init__(self):
        self.__observers = []
        
    def register(self, observer):
        self.__observers.append(observer)
        
    def notifyAll(self, *args, **kwargs):
        for observer in self.__observers:
            observer.notify(self, *args, **kwargs)
            
class Observer1:
    
    def __init__(self, subject):
        subject.register(self)
        
    def notify(self, subject, *args):
        print(f'{type(self).__name__}: Got {args} from {subject}.')
        
class Observer2:
    
    def __init__(self, subject):
        subject.register(self)
        
    def notify(self, subject, *args):
        print(f'{type(self).__name__}: Got {args} from {subject}.')
        
subject = Subject()
obs1 = Observer1(subject)
obs2 = Observer2(subject)

subject.notifyAll('Notification')

Observer1: Got ('Notification',) from <__main__.Subject object at 0x7f001c1ddc70>.
Observer2: Got ('Notification',) from <__main__.Subject object at 0x7f001c1ddc70>.


------------------
## **Observer pattern UML**

- As discussed observer design pattern has two main actors, but we will see three in the UML.
    - *Subject:* aware of the *Observer*. This actor has methods such as *register()* and *deregister()* used by *Observers* to register themselves with *Subject* class. One *Subject* can handle multiple observers.
    
    - *Observer:* defines an interface for objects that are observing the *Subject*. It defines methods that need to be implemented by the *Observer* to get notified of changes in the *Subject*.
    
    - *ConcreteObserver:* stores the state that should be consistent with that of the *Subject*'s state. It implements the *Observer* interface to keep the state consistent with changes in the *Subject*.
![Observer pattern UML](images/observer_pattern.jpg)

--------
# **Observer pattern in real world**

- In this example we will take up a news agency case to get a taste of real-world scenario for the Observer pattern. 
- News agencies typically gather news from various locations and publish them to the subscribers. Let's look at the design considerations for this use case.
    - Information are sent/received in real time. So the agency should be able to publish the news as soon as possible
    - The news can be sent in different channels, such as SMS, mobile, e-mail and so on. And we need to be able to add a new type of subscriber if we add another kind of channel.
    
    
### **Subject design:**
- **Subject** will be the class **NewsPublisher**.
- **NewsPublisher** provides you with an interface so that subscribes can work with it.
- The *attach()* method is used by the **Observer** to subscribe with **NewsPublisher** and *detach()* methods you unsubscribe the **Observer**.
- The *subscriber()* method returns the list of all subscribers wit in **NewsPublisher**.
- The *addNews()* method is used by publisher to create news and *getNews()* return the latest news, which is then notified to the *Observer*.


In [2]:
class NewsPublisher:
    def __init__(self):
        self.__subscribers = []
        self.latestNews = None
        
    def attach(self, subscriber):
        self.__subscribers.append(subscriber)
        
    def detach(self):
        return self.__subscribers.pop()
    
    def subscribers(self):
        return [type(x).__name__ for x in self.__subscribers]
    
    def notifySubscribers(self):
        for sub in self.__subscribers:
            sub.update()
            
    def addNews(self, news):
        self.__latestNews = news
        
    def getNews(self):
        return f'Got news! {self.__latestNews}'
        
        

### **Observer interface design:**

- The **Observer** is **Subscriber**. It's also an abstract base class and represents any other **ConcreteObserver**.
- **Subscriber** has the *update()* method that needs to be implemented by **ConcreteObservers**.
- The *update()* method is implemented by **ConcreteObserver** so they get notified by the **Subject** (**NewsPublishers**) about any news getting published.


In [3]:
from abc import ABCMeta, abstractmethod

class Subscriber(metaclass=ABCMeta):
    @abstractmethod
    def update(self):
        pass

### **ConcreteObserver design:** 

- We have two main observers: **EmailSubscriber** and **SMSSubscriber**. They will implement the subscribe interface.
- In addition to these concrete observers we can have a generic **AnyOtherObserver** observer to demonstrate the loose coupling of observers in the subject.
- The *\_\_init\_\_()* method of each **ConcreteObserver** registers them with **NewsPublisher** with the *attach()* method.
- The *update()* method of **ConcreteObserver** is used internally by **NewsPublisher** to notify about the news additions.

In [4]:
class SMSSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)
        
    def update(self):
        print(f'{type(self).__name__}, {self.publisher.getNews()}')
        
class EmailSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)
        
    def update(self):
        print(f'{type(self).__name__}, {self.publisher.getNews()}')   
        
        
class AnyOtherSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)
        
    def update(self):
        print(f'{type(self).__name__}, {self.publisher.getNews()}')
    

### **What is expected?**
- The client should create an object for **NewsPublisher** that is used by **ConcreteObservers** for many operations.

- **SMSSubscriber**, **EmailSubscriber** and **AnyOtherSubscriber** classes are initialized with publisher objects.

- In the **ConcreteObserver** we will use the *\_\_init\_\_()* method to internally use the *attach()* method of **NewsPublisher** to register itself for news updates.

- We then print the list of all the subscribers (**ConcreteObservers**) that got registered with the **Subject**.

- The object of **NewsPublisher** (*news_publisher*) is then used to create news with the *addNews()* method.

- The *notifySubscribers()* method of **NewsPublisher** is used to notify all subscriber of the news addition. The *notifySubscribers()* method internally calls the *update()* method implemented by **ConcreteObserver** so that they get the latest news.

- **NewsPublisher** also has the *detach()* method to remove the subscriber from the list of subscribers.

In [5]:
if __name__ == '__main__':
    news_publisher = NewsPublisher()
    
    for subs in [SMSSubscriber, EmailSubscriber, AnyOtherSubscriber]:
        subs(news_publisher)
    print(f'Subscribers: {news_publisher.subscribers()}')
        
    news_publisher.addNews('HelloWorld!')
    news_publisher.notifySubscribers()
        
    print(f'Detached {type(news_publisher.detach()).__name__}')
    print(f'Subscribers: {type(news_publisher.subscribers())}')
        
        
    news_publisher.addNews('My second News!!!')
    news_publisher.notifySubscribers()


Subscribers: ['SMSSubscriber', 'EmailSubscriber', 'AnyOtherSubscriber']
SMSSubscriber, Got news! HelloWorld!
EmailSubscriber, Got news! HelloWorld!
AnyOtherSubscriber, Got news! HelloWorld!
Detached AnyOtherSubscriber
Subscribers: <class 'list'>
SMSSubscriber, Got news! My second News!!!
EmailSubscriber, Got news! My second News!!!


---
# **The observer pattern methods**
- There are two different ways of notifying the **Observer** about the changes that happen in the **Subject**. We can classify them as push or pull models.


## **Pull model**
- **Observers** play an active role as follows:
    - The **Subject** broadcasts to all the registered **Observers** when there is any change.
    - The **Observer** is responsible for getting the changes or pulling data from the subscriber when there is an amendment.
    - The pull model is ineffective as it involves two steps:
        - **The first:** where the **Subject** notifies the **Observer**.
        - **The second:** where the **Observer** pulls the required data from **Subject**.
        
## **Push model**
- **Subject** plays active role as follows:
    - The changes are pushed by the **Subject** to the **Observer**.
    - The **Subject** can send detailed information to the **Observer** even if is not needed. This can result in sluggish response times when a large amount of data is sent by the **Subject** bit is never actually used by the **Observer**.
    - Only the required data is sent from the **Subject** to improve performance.
    

---
# **Loose coupling and the observer pattern**

- Loose coupling is an important design principle that should be used in software applications. The main purpose of loose coupling is to strive for loosely-coupled designs between objects that intereact with each other. Coupling refers to the degree of knoowledge that one object has about the other that interacts with it.

- Loosely-coupled design allow us to build flexible object-oriented systems that can handle changes because they reduce the dependency between multiple objects.

- The lose coupling architecture ensures the following features:
    - It reduces the risk that a change made within one element might create an unannticipated impact on the other elements.
    - It simplifies testing, maintenance and troubleshooting.
    - The system can be easily broken down into definable elements.
    
- The observer pattern provides you with an object design where the **Subject** and the **Observer** are loosely coupled. The following points will explain this better:
    - The only thing that the **Subject** knows about an **Observer** is that it implements a certain interface. It need not know the **ConcreteObserver** class.
    - Any new **Observer** can be added at any point in time.
    - The **Subject** need not be modified at all to add any new **Observer**. In the example we sall that **AnyOtherObserver** can be added or removed without any changes in the **Subject**.
    - **Subjects** or **Observers** are not tied up and can be used independently of each other. So the **Observer** can be reused anywhere else, if needed.
    - Changes in the **Subject** or **Observer** will not affect each other. As both are independent or loosely coupled, they are free to make their own changes.
        

---
# **Advantages and disadvantages**

- **<font color = blue> Advantages: </font>**
    - It supports the principle of loose coupling between objects that interact with each other.
    - It allows sending data to other objects effectively without any change in the **Subject** or **Observer** classes.
    - **Observers** can be added or removed at any point in time.

- **<font color = red> Advantages: </font>**
    - The **Observer** interface has to be implemented by **ConcreteObserver**, which involves inheritance. There is no option for composition as the **Observer** interface can be instantiated.
    - If not correctly implemented, the **Observer** can add complexity and lead to inadvertent performance issues.
    - In software application, notifications can sometimes be undependable and result in race conditions or inconsistency.    

---
# **FAQ**

**Q1** Can there be many **Subjects** and **Observers**?
- Yes but to work **Observers** need to be notified of changes in their respectives **Subscribers**.

**Q2** Who is responsible for triggering the update?
- Remember that we have the push and pull models. Typically the **Subject** triggers the update method when there are changes (*push*) or the **Observer** does this (*pull*)

**Q3** Can **Subject** or **Observer** be used for access for any other use case?
- Yes. That is the power here.